# 第6章: 先見（ステップバイステップで考える）

- [レッスン](#レッスン)
- [演習](#演習)
- [サンプルプレイグラウンド](#サンプルプレイグラウンド)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
import json

# utilsパッケージからhintsモジュールをインポート
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

誰かがあなたを起こして、すぐに複数の複雑な質問をし、すぐに答えなければならないとしたら、どうでしょうか？おそらく、**まず答えを考える時間**をもらえた場合ほど上手くいかないでしょう。

予想通り、Claudeも同じです。

**Claudeにステップバイステップで考える時間を与えると、時にClaudeの精度が向上します**。特に複雑なタスクの場合です。ただし、**考えることは声に出した場合にのみ意味があります**。Claudeに考えるように頼んでも、答えだけを出力するように言うことはできません - この場合、実際には考えは行われていません。

### 例

以下のプロンプトでは、人間の読者には2つ目の文が1つ目の文を否定していることが明らかです。しかし、**Claudeは「無関係」という言葉を文字通りに受け取りすぎています**。

In [ ]:
# プロンプト
PROMPT = """この映画レビューの感情はポジティブですか、ネガティブですか？

この映画は新鮮さと独創性で私の心を吹き飛ばしました。全く無関係の話ですが、私は1900年以来岩の下で生活しています。"""

# Claudeの応答を表示
print(get_completion(PROMPT))

Claudeの応答を改善するために、**Claudeに答える前にまず考えさせてみましょう**。そのために、Claudeがタスクを処理し考えるべき手順を文字通り書き出します。ロールプロンプティングを少し加えることで、Claudeはレビューをより深く理解できるようになります。

In [ ]:
# システムプロンプト
SYSTEM_PROMPT = "あなたは映画レビューの賢明な読者です。"

# プロンプト
PROMPT = """このレビューの感情はポジティブですか、ネガティブですか？まず、<positive-argument>と<negative-argument>のXMLタグ内にそれぞれの側の最良の議論を書き、その後答えてください。

この映画は新鮮さと独創性で私の心を吹き飛ばしました。全く無関係の話ですが、私は1900年以来岩の下で生活しています。"""

# Claudeの応答を表示
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claudeは時に順序に敏感です**。この例はClaudeがニュアンスのあるテキストを理解する能力の限界にあり、前の例から議論の順序を入れ替えてネガティブを先にしてポジティブを後にすると、Claudeの全体的な評価がポジティブに変わります。

ほとんどの状況で（ただし全てではなく、混乱するかもしれませんが）、**Claudeは2つのオプションのうち2番目を選ぶ可能性が高くなります**。おそらく、Webからのトレーニングデータでは、2番目のオプションが正解である可能性が高かったためです。

In [ ]:
# プロンプト
PROMPT = """このレビューの感情はネガティブですか、ポジティブですか？まず、<negative-argument>と<positive-argument>のXMLタグ内にそれぞれの側の最良の議論を書き、その後答えてください。

この映画は新鮮さと独創性で私の心を吹き飛ばしました。無関係ですが、私は1900年以来岩の下で生活しています。"""

# Claudeの応答を表示
print(get_completion(PROMPT))

**Claudeに考えさせることで、Claudeの答えを不正確から正確に変えることができます**。Claudeが間違いを犯す多くの場合、それは本当にこれだけのシンプルな方法で修正できるのです！

Claudeの答えが不正確な例を見て、Claudeに考えさせることでどのように修正できるかを見てみましょう。

In [ ]:
# プロンプト
PROMPT = "1956年生まれの俳優が主演する有名な映画の名前を挙げてください。"

# Claudeの応答を表示
print(get_completion(PROMPT))

これを修正するために、Claudeにステップバイステップで考えるように頼みましょう。今回は`<brainstorm>`タグを使用します。

In [ ]:
# プロンプト
PROMPT = "1956年生まれの俳優が主演する有名な映画の名前を挙げてください。まず、いくつかの俳優とその生年について<brainstorm>タグ内でブレインストーミングし、その後答えを出してください。"

# Claudeの応答を表示
print(get_completion(PROMPT))

上記のコンテンツを変更せずにレッスンのプロンプトを試してみたい場合は、レッスンノートブックの一番下までスクロールして[**サンプルプレイグラウンド**](#サンプルプレイグラウンド)をご覧ください。

---

## 演習
- [演習 6.1 - メールの分類](#演習-61---メールの分類)
- [演習 6.2 - メール分類のフォーマット](#演習-62---メール分類のフォーマット)

### 演習 6.1 - メールの分類
この演習では、Claudeにメールを以下のカテゴリーに分類するよう指示します：										
- (A) 購入前の質問
- (B) 破損または欠陥のある商品
- (C) 請求に関する質問
- (D) その他（説明してください）

演習の最初の部分では、`PROMPT`を変更して**Claudeに正しい分類のみを出力させます**。あなたの回答には、**正しい選択肢の文字（A - D）を括弧付きで含め、カテゴリーの名前も含める必要があります**。

`EMAILS`リストの各メールの横にあるコメントを参照して、そのメールがどのカテゴリーに分類されるべきかを確認してください。

In [ ]:
# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = """このメールを緑または青に分類してください：{email}"""

# Claudeの応答の事前入力（ある場合）
PREFILL = ""

# リストとして保存された変数コンテンツ
EMAILS = [
    "こんにちは - 私のMixmaster4000を操作すると奇妙な音がします。また、電子機器が焼けるような、煙っぽいプラスチックのような臭いがします。交換が必要です。", # (B) 破損または欠陥のある商品
    "Mixmaster 4000を使って塗料を混ぜることはできますか？それとも食品を混ぜるだけのものですか？", # (A) 購入前の質問 または (D) その他（説明してください）
    "キャンセルしてから4ヶ月も月額料金が終わるのを待っています！！何が起こっているんだ？？？", # (C) 請求に関する質問
    "ここにどうやって来たのかわかりません。コンピューターが苦手です。助けて。" # (D) その他（説明してください）
]

# 1つのメールに対して複数の正しい分類の可能性を考慮して、リストのリストとして保存された正しい分類
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 正規表現のグレーディングに使用する各カテゴリーの文字列値の辞書
REGEX_CATEGORIES = {
    "A": "A",
    "B": "B",
    "C": "C",
    "D": "D"
}

# メールのリストを反復処理
for i,email in enumerate(EMAILS):
    
    # メールテキストをメールプレースホルダー変数に代入
    formatted_prompt = PROMPT.format(email=email)
   
    # Claudeの応答を取得
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claudeの応答を評価
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Claudeの応答を表示
    print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
    print("ユーザーターン")
    print(formatted_prompt)
    print("\nアシスタントターン")
    print(PREFILL)
    print("\n------------------------------------- Claudeの応答 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 評価 ------------------------------------------")
    print("この演習は正しく解かれました：", grade, "\n\n\n\n\n\n")

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_6_1_hint)

まだ困っていますか？以下のセルを実行して解答例を確認してください。						

In [ ]:
print(hints.exercise_6_1_solution)

### 演習 6.2 - メール分類のフォーマット
この演習では、上記のプロンプトの出力を洗練させて、私たちが望むとおりにフォーマットされた回答を得ます。

お気に入りの出力フォーマット技術を使用して、Claudeに正しい分類の文字だけを`<answer></answer>`タグで囲ませてください。例えば、最初のメールの回答には正確に`<answer>B</answer>`という文字列が含まれるはずです。

各メールに対して正しい文字カテゴリーを忘れた場合は、`EMAILS`リストの横にあるコメントを参照してください。

In [ ]:
# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = """このメールを緑または青に分類してください：{email}"""

# Claudeの応答の事前入力（ある場合）
PREFILL = ""

# リストとして保存された変数コンテンツ
EMAILS = [
    "こんにちは - 私のMixmaster4000を操作すると奇妙な音がします。また、電子機器が焼けるような、煙っぽいプラスチックのような臭いがします。交換が必要です。", # (B) 破損または欠陥のある商品
    "Mixmaster 4000を使って塗料を混ぜることはできますか？それとも食品を混ぜるだけのものですか？", # (A) 購入前の質問 または (D) その他（説明してください）
    "キャンセルしてから4ヶ月も月額料金が終わるのを待っています！！何が起こっているんだ？？？", # (C) 請求に関する質問
    "ここにどうやって来たのかわかりません。コンピューターが苦手です。助けて。" # (D) その他（説明してください）
]

# 1つのメールに対して複数の正しい分類の可能性を考慮して、リストのリストとして保存された正しい分類
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 正規表現のグレーディングに使用する各カテゴリーの文字列値の辞書
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# メールのリストを反復処理
for i,email in enumerate(EMAILS):
    
    # メールテキストをメールプレースホルダー変数に代入
    formatted_prompt = PROMPT.format(email=email)
   
    # Claudeの応答を取得
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claudeの応答を評価
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Claudeの応答を表示
    print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
    print("ユーザーターン")
    print(formatted_prompt)
    print("\nアシスタントターン")
    print(PREFILL)
    print("\n------------------------------------- Claudeの応答 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 評価 ------------------------------------------")
    print("この演習は正しく解かれました：", grade, "\n\n\n\n\n\n")

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_6_2_hint)

### おめでとうございます！

ここまですべての演習を解いたなら、次の章に進む準備ができています。プロンプトエンジニアリングを楽しんでください！

---

## サンプルプレイグラウンド

ここは、このレッスンで示されたプロンプトの例を自由に試したり、プロンプトを微調整してClaudeの応答にどのような影響があるかを確認したりできる領域です。

In [ ]:
# プロンプト
PROMPT = """この映画レビューの感情はポジティブですか、ネガティブですか？

この映画は新鮮さと独創性で私の心を吹き飛ばしました。全く無関係の話ですが、私は1900年以来岩の下で生活しています。"""

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# システムプロンプト
SYSTEM_PROMPT = "あなたは映画レビューの賢明な読者です。"

# プロンプト
PROMPT = """このレビューの感情はポジティブですか、ネガティブですか？まず、<positive-argument>と<negative-argument>のXMLタグ内にそれぞれの側の最良の議論を書き、その後答えてください。

この映画は新鮮さと独創性で私の心を吹き飛ばしました。全く無関係の話ですが、私は1900年以来岩の下で生活しています。"""

# Claudeの応答を表示
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# プロンプト
PROMPT = """このレビューの感情はネガティブですか、ポジティブですか？まず、<negative-argument>と<positive-argument>のXMLタグ内にそれぞれの側の最良の議論を書き、その後答えてください。

この映画は新鮮さと独創性で私の心を吹き飛ばしました。無関係ですが、私は1900年以来岩の下で生活しています。"""

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "1956年生まれの俳優が主演する有名な映画の名前を挙げてください。"

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "1956年生まれの俳優が主演する有名な映画の名前を挙げてください。まず、いくつかの俳優とその生年について<brainstorm>タグ内でブレインストーミングし、その後答えを出してください。"

# Claudeの応答を表示
print(get_completion(PROMPT))